In [8]:
# Robust Fixed Colab Setup Cell

# 1. Setup Environment and Clone Repositories
!git clone https://github.com/KimMeen/Time-LLM.git
!pip install einops transformers accelerate sentencepiece protobuf tabulate scikit-learn -q

# 2. Create Project Structure
import os
os.makedirs("data", exist_ok=True )

# 3. Create HSQP Tokenizer (ROBUST DIMENSION FIX)
with open("hsqp_tokenizer.py", "w") as f:
    f.write("""
import torch
import torch.nn as nn
from einops import rearrange

class HSQP_Tokenizer(nn.Module):
    def __init__(self, seq_len, patch_len, stride, d_model, n_quant_bins=100):
        super(HSQP_Tokenizer, self).__init__()
        self.seq_len = seq_len
        self.patch_len = patch_len
        self.stride = stride
        self.n_quant_bins = n_quant_bins
        self.padding_patch_layer = nn.ReplicationPad1d((0, stride))
        self.stat_proj = nn.Linear(4, d_model // 4)
        self.quant_embed = nn.Embedding(n_quant_bins, d_model // 4)
        self.patch_embed = nn.Linear(patch_len, d_model // 2)
        self.final_proj = nn.Linear(d_model, d_model)

    def quantize(self, x, n_bins=100):
        x_min = x.min(dim=-1, keepdim=True)[0]
        x_max = x.max(dim=-1, keepdim=True)[0]
        x_norm = (x - x_min) / (x_max - x_min + 1e-8)
        return torch.clamp((x_norm * (n_bins - 1)).long(), 0, n_bins - 1)

    def forward(self, x):
        # Ensure x is float right at the start of the method
        x = x.float()

        B, N, L = x.shape

        x = self.padding_patch_layer(x)
        patches = x.unfold(dimension=-1, size=self.patch_len, step=self.stride)
        patches = rearrange(patches, 'b n p l -> (b n) p l')

        # 1. Statistical Features
        m_mean = patches.mean(dim=-1)
        m_std = patches.std(dim=-1)
        m_min = patches.min(dim=-1)[0]
        m_max = patches.max(dim=-1)[0]

        stats = torch.stack([m_mean, m_std, m_min, m_max], dim=-1)
        stat_embed = self.stat_proj(stats)

        # 2. Quantization Features
        quant_indices = self.quantize(patches, self.n_quant_bins)
        # Convert quant_indices to float before calling torch.mode, then back to long for embedding
        quant_indices_float = quant_indices.float()
        quant_mode_float_values = torch.mode(quant_indices_float, dim=-1).values
        quant_mode = quant_mode_float_values.long()
        quant_embed = self.quant_embed(quant_mode)

        # 3. Patch Embeddings
        patch_embed = self.patch_embed(patches)

        combined = torch.cat([stat_embed, quant_embed, patch_embed], dim=-1)

        return self.final_proj(combined), N

"""
)

# 4. Create Modified Model
with open("timellm_hsqp.py", "w") as f:
    f.write("""
import sys
import os
sys.path.append(os.path.abspath("Time-LLM"))
sys.path.append(os.path.abspath("Time-LLM/models"))
sys.path.append(os.path.abspath("Time-LLM/layers"))

import torch
import torch.nn as nn
from models.TimeLLM import Model as TimeLLM_Base
from hsqp_tokenizer import HSQP_Tokenizer

class TimeLLM_HSQP(TimeLLM_Base):
    def __init__(self, configs, patch_len=16, stride=8, use_hsqp=False):
        super().__init__(configs, patch_len, stride)
        self.use_hsqp = use_hsqp
        if self.use_hsqp:
            self.hsqp_tokenizer = HSQP_Tokenizer(configs.seq_len, self.patch_len, self.stride, configs.d_model)

    def forecast(self, x_enc, x_mark_enc, x_dec, x_mark_dec):
        x_enc = self.normalize_layers(x_enc, 'norm')
        B, T, N = x_enc.size()
        x_enc_reshaped = x_enc.permute(0, 2, 1).contiguous().reshape(B * N, T, 1)
        min_values, max_values = x_enc_reshaped.min(1)[0], x_enc_reshaped.max(1)[0]
        medians = x_enc_reshaped.median(1).values
        lags = self.calcute_lags(x_enc_reshaped)
        trends = x_enc_reshaped.diff(dim=1).sum(dim=1)
        prompt = [f"<|start_prompt|>Dataset description: {self.description}Task description: forecast the next {self.pred_len} steps; Input statistics: min {min_values[b].item()}, max {max_values[b].item()}, median {medians[b].item()}, trend {'up' if trends[b]>0 else 'down'}, lags {lags[b].tolist()}<|<end_prompt>|>" for b in range(B*N)]
        x_enc = x_enc_reshaped.reshape(B, N, T).permute(0, 2, 1).contiguous()
        prompt = self.tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=2048).input_ids.to(x_enc.device)
        prompt_embeddings = self.llm_model.get_input_embeddings()(prompt)
        source_embeddings = self.mapping_layer(self.word_embeddings.permute(1, 0)).permute(1, 0)
        x_enc = x_enc.permute(0, 2, 1).contiguous()
        enc_out, n_vars = self.hsqp_tokenizer(x_enc) if self.use_hsqp else self.patch_embedding(x_enc)
        enc_out = self.reprogramming_layer(enc_out, source_embeddings, source_embeddings)
        dec_out = self.llm_model(inputs_embeds=torch.cat([prompt_embeddings, enc_out], 1)).last_hidden_state[:, :, :self.d_ff]
        dec_out = torch.reshape(dec_out, (-1, n_vars, dec_out.shape[-2], dec_out.shape[-1])).permute(0, 1, 3, 2).contiguous()
        dec_out = self.output_projection(dec_out[:, :, :, -self.patch_nums:]).permute(0, 2, 1).contiguous()
        return self.normalize_layers(dec_out, 'denorm')
"""
)

# 5. Create Data Utils
with open("data_utils.py", "w") as f:
    f.write("""
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler

class TimeSeriesDataset(Dataset):
    def __init__(self, data, seq_len, pred_len):
        self.seq_len, self.pred_len, self.data = seq_len, pred_len, data
        self.x, self.y = [], []
        for i in range(len(data) - seq_len - pred_len + 1):
            self.x.append(data[i:i+seq_len])
            self.y.append(data[i+seq_len:i+seq_len+pred_len])
    def __len__(self): return len(self.x)
    def __getitem__(self, i): return torch.tensor(self.x[i]).float(), torch.zeros(self.seq_len, 1), torch.tensor(self.y[i]).float(), torch.zeros(self.pred_len, 1)

def load_data(name, path, seq_len, pred_len):
    df = pd.read_csv(path)
    df_data = df.iloc[:, 1:].values.astype(np.float32)
    n_vars = df_data.shape[1]
    tr_sz, val_sz = int(len(df_data)*0.7), int(len(df_data)*0.1)
    scaler = StandardScaler()
    train = scaler.fit_transform(df_data[:tr_sz])
    val = scaler.transform(df_data[tr_sz:tr_sz+val_sz])
    test = scaler.transform(df_data[tr_sz+val_sz:])
    return TimeSeriesDataset(train, seq_len, pred_len), TimeSeriesDataset(val, seq_len, pred_len), TimeSeriesDataset(test, seq_len, pred_len), n_vars, train
"""
)

# 6. Create Metrics
with open("metrics.py", "w") as f:
    f.write("""
import numpy as np
def compute_metrics(pred, true, hist):
    mse = np.mean((pred - true)**2)
    mae = np.mean(np.abs(pred - true))
    h_rate = np.mean((pred < hist.min() - 0.2*(hist.max()-hist.min())) | (pred > hist.max() + 0.2*(hist.max()-hist.min()))) * 100
    var = np.var(pred - true) + 1e-8
    perp = np.exp(0.5 * np.log(2 * np.pi * var) + 0.5)
    return {"MSE": mse, "MAE": mae, "Hallucination": h_rate, "Perplexity": perp}
"""
)

# 7. Download Datasets
#!wget -q https://raw.githubusercontent.com/zhouhaoyi/ETDataset/main/ETT-small/ETTh1.csv -O data/ETTh1.csv
#!wget -q https://raw.githubusercontent.com/zhouhaoyi/ETDataset/main/ETT-small/ETTh2.csv -O data/ETTh2.csv
#!wget -q https://raw.githubusercontent.com/zhouhaoyi/ETDataset/main/ETT-small/ETTm1.csv -O data/ETTm1.csv
#!wget -q https://raw.githubusercontent.com/zhouhaoyi/ETDataset/main/ETT-small/ETTm2.csv -O data/ETTm2.csv
!wget -q https://huggingface.co/datasets/pkr7098/time-series-forecasting-datasets/resolve/main/weather.csv -O data/weather.csv
#!wget -q https://huggingface.co/datasets/pkr7098/time-series-forecasting-datasets/resolve/main/traffic.csv -O data/traffic.csv
!wget -q https://huggingface.co/datasets/pkr7098/time-series-forecasting-datasets/resolve/main/national_illness.csv -O data/traffic.csv
print("Setup Complete! You can now run the experiment cell." )


fatal: destination path 'Time-LLM' already exists and is not an empty directory.
Setup Complete! You can now run the experiment cell.


In [10]:
# Updated Colab Experiment Run Cell

import torch
from torch.utils.data import DataLoader
import sys
import os
import numpy as np
import pandas as pd
from tabulate import tabulate
import warnings

# Ensure the current directory is in sys.path for local imports
sys.path.insert(0, ".")

from timellm_hsqp import TimeLLM_HSQP
from data_utils import load_data
from metrics import compute_metrics

warnings.filterwarnings('ignore')

class Config:
    def __init__(self, **kwargs):
        self.task_name = 'long_term_forecast'
        self.prompt_domain = False
        self.content = ""

        self.seq_len = 96
        self.pred_len = 192

        self.patch_len = 16
        self.stride = 16

        self.d_model = 64
        self.d_ff = 256
        self.n_heads = 4
        self.dropout = 0.1
        self.head_dropout = 0.1

        self.llm_model = 'GPT2'
        self.llm_layers = 1
        self.llm_dim = 768

        self.enc_in = 7

        self.batch_size = 512
        self.epochs = 5
        self.learning_rate = 1e-4

        # Overwrite defaults with any provided keyword arguments
        for k, v in kwargs.items():
            setattr(self, k, v)

# Configuration for the experiments
config = Config(
    epochs=2,        # Set to 10 or 20 for final results
    llm_layers=1,    # Set to 2 or more for better accuracy
    batch_size=64
)

datasets = {
    #'ETTh1': 'data/ETTh1.csv', 'ETTh2': 'data/ETTh2.csv',
    #'ETTm1': 'data/ETTm1.csv', 'ETTm2': 'data/ETTm2.csv',
    'Weather': 'data/weather.csv',
    'national_illness': 'data/national_illness.csv'

}

results = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running experiments on: {device}")

for name, path in datasets.items():
    if not os.path.exists(path):
        print(f"Skipping {name}: File not found at {path}")
        continue

    print(f"\nProcessing {name}...")
    train_set, val_set, test_set, n_vars, hist_data = load_data(name, path, config.seq_len, config.pred_len)

    for use_hsqp in [False, True]:
        model_name = "TimeLLM+HSQP" if use_hsqp else "TimeLLM"
        print(f"  Running {model_name}...")

        # Create a new config object for each model and update enc_in
        model_config = Config(**vars(config))
        model_config.enc_in = n_vars

        model = TimeLLM_HSQP(model_config, use_hsqp=use_hsqp).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
        criterion = torch.nn.MSELoss()

        # Simple training loop
        model.train()
        train_loader = DataLoader(train_set, batch_size=config.batch_size, shuffle=True, num_workers=2)
        for epoch in range(config.epochs):
            for x, xm, y, ym in train_loader:
                optimizer.zero_grad()
                pred = model(x.to(device), xm.to(device), y.to(device), ym.to(device))
                loss = criterion(pred, y.to(device))
                loss.backward()
                optimizer.step()

        # Evaluation
        model.eval()
        test_loader = DataLoader(test_set, batch_size=config.batch_size, num_workers=2)
        preds, trues = [], []
        with torch.no_grad():
            for x, xm, y, ym in test_loader:
                preds.append(model(x.to(device), xm.to(device), y.to(device), ym.to(device)).cpu().numpy())
                trues.append(y.numpy())

        m = compute_metrics(np.concatenate(preds), np.concatenate(trues), hist_data)
        results.append([name, model_name, f"{m['MSE']:.4f}", f"{m['MAE']:.4f}", f"{m['Hallucination']:.2f}%", f"{m['Perplexity']:.4f}"])

print("\n" + tabulate(results, headers=["Dataset", "Model", "MSE", "MAE", "Hallucination", "Perplexity"], tablefmt="grid"))


AttributeError: partially initialized module 'torch' has no attribute 'jit' (most likely due to a circular import)